##Citibike NYC Analysis 2017
[Citibike System Data](https://www.citibikenyc.com/system-data)  
[Download Citibike trip history data](https://s3.amazonaws.com/tripdata/index.html)

In the following cell below, replace {YOUR STORAGE ACCOUNT NAME} with the Azure storage account name you created and {YOUR STORAGE ACCOUNT ACCESS KEY} with your storage account access key.

In [3]:
spark.conf.set("fs.azure.account.key.{YOUR STORAGE ACCOUNT NAME}.blob.core.windows.net", "{YOUR STORAGE ACCOUNT ACCESS KEY}")

In the following cell below, replace {YOUR STORAGE ACCOUNT NAME} with the Azure storage account name you created and {YOUR CONTAINER NAME} where you have uploaded the data (csv files).

In [5]:
%sql 
drop table if exists citibike_data;
create temporary table citibike_data
using csv
options (
 path "wasbs://{YOUR CONTAINER NAME} @{YOUR STORAGE ACCOUNT NAME}.blob.core.windows.net/ny/csv/2017/*.csv", header "true"
);

Let's see how many bike trips were taken in 2017.

In [7]:
%sql
select count (*) from citibike_data;

Turns out there were 16,364,657 trips taken in 2017.

Here is a quick look at the columns (attributes) from the dataset (limit 100)

In [10]:
%sql
select * from citibike_data limit 100;

How many citibikes are there in NYC?

In [12]:
%sql
select count(distinct(bikeid)) as number_of_bikes
from citibike_data;

A total of 14,204 citibikes were available in NYC in 2017.

With more than 819 citibike stations let's determine the top 20 most popular stations when starting a route (trip).

In [15]:
%sql
select  `start station name`,
        `start station latitude`,
        `start station longitude`,
        count (*) as number_of_trips
from    citibike_data
group by
        `start station name`,
        `start station latitude`,
        `start station longitude`
order by
        number_of_trips desc
limit 20;

Each trip in the citibike dataset includes a usertype column indicating whether the rider is a subscriber or a customer (non-subscriber). How many trips have been taken by subscribers in 2017, and what is the difference in average in trip-duration for subscribers vs. non-subscribers.

In [17]:
%sql
select  usertype,
        count(*) as num_of_trips, 
        round(avg(cast(tripduration as int) / 60), 2) as duration
from    citibike_data
where   usertype in ('Subscriber', 'Customer')
group by
        usertype; 

Turns out that 89% of trips are taken by subscribers, but their average trip duration is significantly less: 14 minutes compared to 42 for non-subscribers. This is most likely because subscribers use the bikes for commuting, whereas non-subscribers use them for sightseeing.

These are the top routes (trips) broken down by subscribers and customers

In [20]:
%sql
select  usertype,  
        concat(`start station name`, " to ", `end station name`) as route,  
        count(*) as number_of_trips,  
        round(avg(cast(tripduration as int) / 60),2) as duration
from    citibike_data
group by  
      `start station name`, 
      `end station name`, 
       usertype
order by  
      number_of_trips desc
limit 20;

The following query returns the number of trips by age and gender for 2017 (limited to less than 90 years old)

In [22]:
%sql
select  (2017 - cast(`birth year` as int)) as age,
        count(*) total_count, 
        count(case when gender = 1 then 1 end) as number_of_males,
        count(case when gender = 2 then 1 end) as number_of_females, 
        count(case when gender = 0 then 1 end) as number_of_unknown
from    citibike_data
where   cast(`birth year` as string) != 'NULL' and
        (2017 - cast(`birth year` as int)) <= 90
group by age
order by age;

Let's explore the bike routes (trips) by gender, to see if they are similar.

In [24]:
%sql
select  concat(`start station name`, " to ", `end station name`) as route,  
        count(*) as number_of_trips 
from    citibike_data
where   gender = 2 --female
group by 
        `start station name`, `end station name`
order by number_of_trips desc
limit 5;

In [25]:
%sql
select  concat(`start station name`, " to ", `end station name`) as route,  
        count(*) as number_of_trips 
from    citibike_data
where   gender = 1 --male
group by 
        `start station name`, `end station name`
order by number_of_trips desc
limit 5;

We can clearly conclude that they are quite diffent, except for one overlap station "E 7 St & Avenue A to Cooper Square & E 7 St" which is the most popular one for both genders.

Every citibike has a unique bike ID (bikeid column), which bike had the most trips in 2017?

In [28]:
%sql
select  bikeid,  
        count(*) as trip_count 
from    citibike_data
group by bikeid
order by trip_count desc
limit 5;

Looks like bikeid 25738 had the most trips...

That's it! Nothing much, but a great way to start exploring and using Azure Databricks to analyse data. if you are interested to go further I would recommend you download all of the citibike data from 2013 to 2017 and walk through the same exercices (minor changes will need to done to the queries).